<a href="https://colab.research.google.com/github/Klaudia2022/Algorytm-eliminacji-Gaussa---Jordana/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
import numpy as np
from scipy.linalg import block_diag

#Zadanie 1
---



In [ ]:
def GaussJordan(A,red=True):
  A=A.astype(float)               #bez tej linijki liczyło mi całkowitoliczbowo, a nie na liczbach rzeczywistych
  m=A.shape[0]
  n=A.shape[1]
  if red==True:
    for i in range(min(m,n)):
      y=i
      while A[y,i]==0 and y<m-1:
        y+=1
      if y==m-1 and A[y,i]==0:
        continue
      else:
        A[y,:]=A[y,:]/A[y,i]
        pom=A[i,:].copy()               #zamieniam wiersze miejscami, nie zmienia to niczego do sprawdzania czy macierz jest osobliwa, ponieważ zamiana wierszy miejscami zmienia tylko znak wyznacznika
        A[i,:]=A[y,:].copy()
        A[y,:]=pom.copy()
        for j in range(m):
          if j==i:
            continue
          else:
            A[j,:]=A[j,:]-A[i,:]*A[j,i]
  else:
    for i in range(min(m,n)):
      y=i
      while A[y,i]==0 and y<m-1:
        y+=1
      if y==m-1 and A[y,i]==0:
        continue
      else:
        x=A[y,:]/A[y,i]
        pom=A[i,:].copy()
        A[i,:]=A[y,:].copy()
        A[y,:]=pom.copy()
      for j in range(i+1, m):
        A[j,:]=A[j,:]-x*A[j,i]
  return A

#Zadanie 2
---

In [ ]:
def rozwiazuklad(A,b, niewiadome=[]):                        #użytkownik może wprowadzić zmienne (symbole)
  m=A.shape[0]
  n=A.shape[1]
  B=np.hstack((A,b))
  B=GaussJordan(B, True)
  r=np.linalg.matrix_rank(B)
  if r==n+1:
    print("Brak rozwiązań")
    return None
  else:
    if niewiadome==[]:
      niewiadome=sp.symbols(['x'+str(i+1) for i in range(n)])
    rozwiazanie={}
    for i in range(r):
      rozwiazanie[niewiadome[i]]=B[i,n]
      for j in range(r+1,n+1):
        rozwiazanie[niewiadome[i]]-=niewiadome[j-1]*B[i,j-1]
    return rozwiazanie

#Zadanie 3
---

In [ ]:
def osobliwa(A):
  if A.shape[0]!=A.shape[1]:
    return None
  A=GaussJordan(A)
  det=1
  for i in range(A.shape[0]):
    det*=A[i,i]
  if det==0:
    return True
  else:
    return False

#Zadanie 4
---

In [ ]:
def odwrotna(A):
  if A.shape[0]!=A.shape[1]:
    return None
  n=A.shape[0]
  if osobliwa(A):
    return None
  else:
    B=np.hstack((A,np.eye(n)))
    B=GaussJordan(B)
    return B[0:n,n:2*n]

#Zadanie 5
---

##a)

In [ ]:
A=np.array([[1,2],[2,-1]])

In [ ]:
b=np.array([[8],[1]])

In [ ]:
rozwiazuklad(A,b)

{x1: 2.0, x2: 3.0}

In [ ]:
zmienne=sp.symbols(['x','y'])

In [ ]:
rozwiazuklad(A,b,zmienne)

{x: 2.0, y: 3.0}

##b)

In [ ]:
A=np.array([[-1,2,1],[1,-3,-2],[3,-1,-1]])

In [ ]:
b=np.array([[-1],[-1],[4]])

In [ ]:
rozwiazuklad(A,b)

{x1: 2.0, x2: -1.0, x3: 3.0}

##c)

In [ ]:
A=np.array([[2,5,7],[6,3,4],[5,-2,-3]])

In [ ]:
C=odwrotna(A)

In [ ]:
C

array([[  1.,  -1.,   1.],
       [-38.,  41., -34.],
       [ 27., -29.,  24.]])

#Porównanie czasu
---

In [ ]:
from timeit import default_timer as timer

In [ ]:
timeMy=0.0
timeBuilt=0.0
size1=25
size2=25
rep=1000

for i in range(rep):
  A=np.random.random((size1,size2))
  b=np.random.random((size1,1))
  start=timer()
  rozwiazuklad(A,b)
  end=timer()
  timeMy+=end-start

  start=timer()
  np.linalg.solve(A,b)
  end=timer()
  timeBuilt+=end-start

print("Funkcja rozwiazuklad: ", (timeMy/rep)*1000, "ms")
print("Funkcja wbudowana: ", (timeBuilt/rep)*1000, "ms")

Funkcja rozwiazuklad:  5.058541706991491 ms
Funkcja wbudowana:  0.15427351999005623 ms


Funkcja osobliwa to nie jest dokładnie to samo co wyznacznik, ale nie znalazłam lepszego odpowiednika

In [ ]:
timeMy=0.0
timeBuilt=0.0
size=100
rep=1000

for i in range(rep):
  A=np.random.random((size,size))
  start=timer()
  osobliwa(A)
  end=timer()
  timeMy+=end-start

  start=timer()
  np.linalg.det(A)
  end=timer()
  timeBuilt+=end-start

print("Funkcja osobliwa: ", (timeMy/rep)*1000, "ms")
print("Funkcja wbudowana: ", (timeBuilt/rep)*1000, "ms")

Funkcja osobliwa:  65.83809302400641 ms
Funkcja wbudowana:  0.4917819850015804 ms


In [ ]:
timeMy=0.0
timeBuilt=0.0
size=100
rep=1000

for i in range(rep):
  A=np.random.random((size,size))
  while np.linalg.det(A)==0:
    A=np.random.random((size, size))
  start=timer()
  odwrotna(A)
  end=timer()
  timeMy+=end-start

  start=timer()
  np.linalg.inv(A)
  end=timer()
  timeBuilt+=end-start

print("Funkcja odwrotna: ", (timeMy/rep)*1000, "ms")
print("Funkcja wbudowana: ", (timeBuilt/rep)*1000, "ms")

Funkcja odwrotna:  69.84931803000063 ms
Funkcja wbudowana:  0.7681545779946646 ms
